# OpenStreetMap Data Case Study
## Map Area: Dallas, United States

https://mapzen.com/data/metro-extracts/metro/dallas_texas/

Having lived in the Dallas area for over 15 years, I am very interested in working with this region.

## Problems Researched and Encountered In The Map

Extracting a subset of the Dallas OSM file using the code provided in the Project Instructions, initially I used a k value of 100 (1% sample) to create CSV files for each of the following:

    Nodes
    Node Tags
    Ways
    Ways Tags
    Ways Nodes

Eventually, to increases the size of the data source, I repeated much of the analysis with a k value of 35 which allowed for a 50MB+ OSM file.  Of most interest was the Node Tags and Ways Tags as these provide a majority of the user provided data that is most subject to error.

The following potential data problems were researched:
1. Erroneous city names.
2. Zip Code accuracy.
3. Abbreviations in Street Names.

This output can be replicated by executing the code within "Create CSV.ipynb," then starting an SQLite3 session and performing the command ".read import_csv.sql" which will remove any previous instances of the dallas.db and reinitialize the database with the current content of the .csv files.  Some of the illustritive information below came from the initial pull (k=100), however, output of a full session performing the embedded SQL statements against the larger data set below will be included in "OSM_SQL.txt".

### 1. Erroneous city names

Using a query shared in the sample by github user carlward, I summarized the values in the city tags.

    SELECT tags.value, COUNT(*) as count 
    FROM (SELECT * FROM nodes_tags UNION ALL 
          SELECT * FROM ways_tags) tags
    WHERE tags.key LIKE '%city'
    GROUP BY tags.value
    ORDER BY count DESC;
    
Surprisingly, this list represented a very high degree of accuracy, however there were two obvious errors involving city names with only numbers.  (This query summary was pulled with sample k value of 100.  Subsequently, a larger sample was pulled with a k value of 35.  A review of that data confirmed the level of accuracy and identified one city with a city name of "15."  Other items that could be resolved involved capitalization.)

|City|Count|
|:---|---|
|Frisco|495|
|Plano|27|
|Fort Worth|8|
|Dallas|7|
|Cedar Hill|4|
|McKinney|4|
|Waxahachie|4|
|Irving|3|
|Wylie|3|
|Grand Prairie|2|
|The Colony|2|
|100|1|
|2|1|
|Arlington|1|
|Corinth|1|
|Denton|1|
|Duncanville|1|
|Euless|1|
|Farmers Branch|1|
|Keller|1|
|Kennedale|1|
|Lewisville|1|
|Little Elm|1|
|Mansfield|1|
|Mesquite|1|
|Nevada|1|
|Palmer|1|
|Richardson|1|
|Rockwall|1|
|Saginaw|1|
|Southlake|1|

Using the following query I was able to isolate only these rows which contain numeric values.

    SELECT tags.value, COUNT(*) as count
    FROM (SELECT * FROM nodes_tags UNION ALL 
          SELECT * FROM ways_tags) tags
    WHERE tags.key LIKE '%city'
    AND tags.value GLOB '*[0-9]*'
    GROUP BY tags.value
    ORDER BY count DESC;

For purposes of this project, the decision was made to clean this data by providing a reliable alph-only value in the City field, and thus instances city fields containing non-alpha characters were simply deleted.  However, given more time and the proper source data, a more thourough cleaning method would involve the comparison of other fields (most notably the zip code) to a reference source and correcting the city using this information.


    UPDATE nodes_tags
    SET value = ''
    WHERE key LIKE '%city'
    AND value GLOB '*[0-9]*';

    UPDATE ways_tags
    SET value = ''
    WHERE key LIKE '%city'
    AND value GLOB '*[0-9]*';
    
To verify updates, the following select was performed and confirmed that no matching data existed in the database.
    
    SELECT tags.value, COUNT(*) as count
    FROM (SELECT * FROM nodes_tags UNION ALL 
          SELECT * FROM ways_tags) tags
    WHERE tags.key LIKE '%city'
    AND tags.value GLOB '*[0-9]*'
    GROUP BY tags.value
    ORDER BY count DESC;


NOTE: One unique situation may affect the accuracy of this "more thorough" cleaning.  In at least one instance in the Dallas area (zip code 76179), there are two possible city mappings (Fort Worth and Saginaw) that could be correct.  Further analysis by street address or GPS coordinates may be required to truly correct the instance of the incorrect city.

### 2. Zip Code Accuracy

Using the following query (slightly modified from the above query taken from the sample) I was able to find all zip codes represented within the sampled list.

    SELECT tags.value, COUNT(*) as count
    FROM (SELECT * FROM nodes_tags UNION ALL 
          SELECT * FROM ways_tags) tags
    WHERE tags.key = 'postcode'
    GROUP BY tags.value
    ORDER BY tags.value;
    
This search actually returned no questionable values.  All zip codes returned were either 750xx, 751xx, 752xx, 760xx, 761xx or 762xx.  Using the following resource (as well as personal knowledge of the area) all of these zip code prefixes are in the area generally referred to as the Dallas (or Dallas-Fort Worth) area.

https://en.wikipedia.org/wiki/List_of_ZIP_code_prefixes#Starts_with_7

However, while the City search on the sampled data returned over 1500 city tags, the zip code search returned fewer than 100 matching tags.  This suggests that while the values may be largely accurate, many of the records appear to be missing this data altogether.

This search would be repeated on gradually larger samples of the data set, each time comparing the summarized results to expected values and looking for obvious outliers.  Examples of outliers here could be:
* Zip codes with fewer than 5 digits.
* Zip codes with more than 5 digits but fewer than 9 digits (including and excluding a possible - character).
* Zip codes with more than 9 digits (excluding any possible - characters).
* Zip codes with values other than digits.

An example of the update queries to remove erroneous zip codes containing non-digits would be:

    UPDATE nodes_tags
    SET value = ''
    WHERE key='%postcode'
    AND value GLOB '*[^0-9]*';
    
    UPDATE ways_tags
    SET value = ''
    WHERE key='postcode'
    AND value GLOB '*[^0-9]*';
    
Because this data set only contained one entry with non-digits, "TX 76262", the correction was obvious and rather than deleting it, the decision was made to directly modify it to its proper value of "76262" using the following query:

    UPDATE nodes_tags
    SET value = '76262'
    WHERE key='postcode'
    AND value = 'TX 76262';
    
To verify the update was successful, the followng query is used before and after the above update query:
    
    SELECT *
    FROM nodes_tags
    WHERE key='postcode'
    AND value GLOB '*[^0-9]*';


An example of the update queries to remove erroneous zip codes with length less than 5 digits would be:
    
    UPDATE nodes_tags
    SET value = ''
    WHERE key='postcode'
    AND LENGTH(value) < 5;
    
    UPDATE ways_tags
    SET value = ''
    WHERE key='postcode'
    AND LENGTH(value) < 5;

An example of the update queries to remove erroneous zip codes with only digits and length longer than 9 characters:

    UPDATE nodes_tags
    SET value = ''
    WHERE key='postcode'
    AND value GLOB '*[0-9]*'
    AND LENGTH(value) > 9;
    
    UPDATE ways_tags
    SET value = ''
    WHERE key='postcode'
    AND value GLOB '*[0-9]*'
    AND LENGTH(value) > 9;
    
An example of the update queries to remove erroneous zip codes with only digits and the - character and length longer than 10 characters:

    UPDATE nodes_tags
    SET value = ''
    WHERE key='postcode'
    AND value GLOB '*[0-9]-[0-9]*'
    AND LENGTH(value) > 10;
    
    UPDATE ways_tags
    SET value = ''
    WHERE key='postcode'
    AND value GLOB '*[0-9]-[0-9]*'
    AND LENGTH(value) > 10;

## 3. Abbreviations In Street Names

Seeing the evidence of inconsistencies in street naming and abbreviations, this seemed a good place to look within this data set.  However, once again, the Dallas OSM data is REMARKABLY clean!  Using the following query I identified 7,922 unique street names in the database:

    SELECT tags.value, COUNT(*) as count
    FROM (SELECT * FROM nodes_tags UNION ALL 
          SELECT * FROM ways_tags) tags
    WHERE tags.key = 'street'
    GROUP BY tags.value
    ORDER BY count DESC;
    
Looking through the results, the first item identified was that in some cases "Avenue B" was entered as "Ave B," for example.  A quick glance through the remaining data showed that abbreviations for directions (North, etc.) were all but non-existent.  In addition, words like "Drive," "Street," etc., were all but perfectly uniform with no abbreviations.  It seems that this data set was either input from an already uniformly formatted source or has already been "wrangled."

## Overview Of The Data

### File Sizes

    dallas_texas.osm ...... 55,136 KB
    dallas.db ............. 31,802 KB
    nodes.csv ............. 22,737 KB
    nodes_tags.csv ........... 363 KB
    ways.csv ............... 1,749 KB
    ways_tags.csv .......... 6,725 KB
    ways_nodes.cv .......... 3,635 KB

### Number of Nodes
    SELECT COUNT(*) FROM nodes;

* 249,683

### Number of Ways
    SELECT COUNT(*) FROM ways;
    
* 25,934

### Number of Unique Users
    SELECT COUNT(DISTINCT(e.uid))          
    FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;

* 1,103

### Top 10 Contributing Users
    SELECT e.user, COUNT(*) as num
    FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
    GROUP BY e.user
    ORDER BY num DESC
    LIMIT 10;
    
    Andrew Matheny_import... 174,102
    woodpeck_fixbot.......... 29,508
    Andrew Matheny............ 7,634
    Stephen214................ 6,339
    TheDude05................. 4,124
    TexasNHD.................. 2,514
    25or6to4.................. 2,429
    RoadGeek_MD99............. 1,808
    Chris Lawrence............ 1,696
    Dami_Tn................... 1,487

### Number of users appearing only once (having 1 post)
    SELECT COUNT(*) 
    FROM
        (SELECT e.user, COUNT(*) as num
         FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
         GROUP BY e.user
         HAVING num=1)  u;

* 338

### Top 10 Described Amenities
    SELECT tags.value, COUNT(*) as count
    FROM (SELECT * FROM nodes_tags UNION ALL 
          SELECT * FROM ways_tags) tags
    WHERE tags.key = 'amenity'
    GROUP BY tags.value
    ORDER BY count DESC
    LIMIT 10;

    parking............ 151
    place_of_worship.... 92
    school.............. 55
    fast_food........... 47
    restaurant.......... 47
    fuel................ 31
    bank................ 14
    bench................ 8
    grave_yard........... 7
    hospital............. 7

### Key Types
    SELECT tags.key, COUNT(*) as count
    FROM (SELECT * FROM nodes_tags UNION ALL 
          SELECT * FROM ways_tags) tags
    GROUP BY tags.key
    ORDER BY count DESC
    LIMIT 10;
    
* 285 different tipes of keys represented.
* Top 10 most common:

        building...... 16,946
        street........ 14,754
        housenumber... 14,753
        highway........ 8,442
        county......... 5,015
        cfcc........... 5,008
        name........... 4,960
        reviewed....... 4,409
        name_base...... 3,428
        source......... 3,292

## Additional Ideas

Other potential areas of investigation, given more time and resources might be:

1. Given more time and the proper source data, a more thourough cleaning method would involve the comparison of other fields (most notably the zip code) to a reference source and correcting the city using this information.
2. Similarly the missing zip code data could be largely remedied by comparison to a city/zip reference table or an available geographic zip code match.
3. Based on the sample data, it is clear that, within the Dallas Fort Worth Metroplex, someone is keenly interested in the accuracy of the Frisco area.  Frisco is one of the most popular residential suburbs in the metroplex, so the robustness of this area's data is not surprising, however, for this OSM data to be remotely reflective of the "Greater Dallas/Fort Worth Metroplex," much more data for the areas other than Frisco is needed.
4. There are many entries in the data labeled "FIXME" indicating that there is a clear need for update to the record.  These records can be identified with the following query:

        SELECT *
        FROM (SELECT * FROM nodes_tags UNION ALL 
              SELECT * FROM ways_tags) tags
        WHERE tags.key LIKE 'fixme';
  
  Further investigation here could be under taken.

However, the problem with this data is much bigger than typographical errors contained within.  The problem is that the commercialization of this information is so advanced now that the real world value of the information collected is questionable when compared with robust tools like WAZE, Google Maps, Yahoo Maps, Apple Maps, etc.  Because these tools have monetized the data in such a way that is it hard to keep up with the quality that is invested in these tools.  The excercise is extremely valuable as used here, as a learning tool because it is so well structured, and while woefully incomplete, it is still robust enough to challenge the student of data science to break out of habits such as using Excel to perform analysis because of limited data sets.